<a href="https://colab.research.google.com/github/arny617/MLNanodegree/blob/master/Crypto_SentimentAnalysis_RandomForest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords as sw
from nltk.tokenize import word_tokenize
from nltk import pos_tag
import numpy as np
import nltk
import os
nltk.download('wordnet')
import pandas as pd
import numpy as np
import pickle
import joblib

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
!pip install pysentiment

     |████████████████████████████████| 2.8MB 12.6MB/s 
ERROR: Command errored out with exit status 1: python setup.py egg_info Check the logs for full command output.


In [4]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import gensim

/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
!pip install OpenBlender

     |████████████████████████████████| 61kB 5.6MB/s 
     |████████████████████████████████| 245kB 19.3MB/s 
  Created wheel for OpenBlender: filename=OpenBlender-2.6-cp36-none-any.whl size=6607 sha256=09fbeaba11c41b8b085d6ae177b6c7e592e54beef3cd5346ad4ad61e9b98d3c5
  Stored in directory: /root/.cache/pip/wheels/fd/29/7b/377afef16a792ed174a9ade5a6707ef87843e214476d0c8205
Successfully built OpenBlender


In [6]:
import OpenBlender

In [7]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import os
from google.colab import drive
drive.mount('/content/gdrive')
ROOT = "/content/gdrive/My Drive/Colab Notebooks/NLP/Fast.ai_IntroToNLP/"
os.chdir(ROOT)

Mounted at /content/gdrive


In [ ]:
api_token = '5fc5ad47951629617230bee5MDHzrVvlWhz1MZm92QeQqc5DrybPBT'

In [ ]:
action = 'API_getObservationsFromDataset'# ANCHOR: 'Bitcoin vs USD'
  
parameters = { 
    'token' : api_token,
    'id_dataset' : '5d4c3af79516290b01c83f51',
    'date_filter':{"start_date" : "2020-01-01",
                   "end_date" : "2020-08-29"} }
df = pd.read_json(json.dumps(OpenBlender.call(action, parameters)['sample']), convert_dates=False, convert_axes=False).sort_values('timestamp', ascending=False)
df.reset_index(drop=True, inplace=True)
df['date'] = [OpenBlender.unixToDate(ts, timezone = 'GMT') for ts in df.timestamp]
df = df.drop('timestamp', axis = 1)

Task ID: '5fc84fb90895fafb4a9d8d4b'.
Total estimated consumption: 5832.61 processing units.
Task confirmed. Starting download..
100.0 % completed.


In [ ]:
df['log_ret'] = np.log(df['price']) - np.log(df['open'])

In [ ]:
df['target'] = [1 if log_diff > 0 else 0 for log_diff in df['log_ret']]

In [ ]:
format = '%d-%m-%Y %H:%M:%S'
timezone = 'GMT'
df['u_timestamp'] = OpenBlender.dateToUnix(df['date'], 
                                           date_format = format, 
                                           timezone = timezone)
df = df[['date', 'u_timestamp', 'price', 'target']]
df.head()

,date,u_timestamp,price,target
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0


In [ ]:
blend_source = {
                'id_dataset':'5ea2039095162936337156c9',
                'feature' : 'text'
            }
# Now, let's 'timeBlend' it to our dataset
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw')
df = pd.concat([df, df_blend.loc[:, df_blend.columns != 'u_timestamp']], axis = 1)
df.head()

Task ID: '5fc850470895fafb4a9d8d4c'.
Total estimated consumption: 9440.0 processing units.
Task confirmed. Starting download..
100%


,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...


In [ ]:
df['text'] = df['BITCOIN_NE.text_last1days'].apply(lambda x:"xxeos ".join(x))

In [ ]:
vader_analyzer = SentimentIntensityAnalyzer()

In [ ]:
df["text_polarity"] = df["text"].apply(lambda x : vader_analyzer.polarity_scores(x))
df["text_subjectivity"] = df["text"].apply(lambda x : TextBlob(x).sentiment[1])

In [ ]:
# BTC Text Vectorizer
blend_source = { 
                'id_textVectorizer':'5f739fe7951629649472e167'
               }
df_blend = OpenBlender.timeBlend( token = api_token,
                                  anchor_ts = df.u_timestamp,
                                  blend_source = blend_source,
                                  blend_type = 'agg_in_intervals',
                                  interval_size = 60 * 60 * 24,
                                  direction = 'time_prior',
                                  interval_output = 'list',
                                  missing_values = 'raw').add_prefix('VEC.')

Task ID: '5fc857840895fafb4a9d8d4c'.
Total estimated consumption: 10479.43 processing units.
Task confirmed. Starting download..
100%


In [ ]:
df_anchor = pd.concat([df, df_blend.loc[:, df_blend.columns != 'timestamp']], axis = 1)
df_anchor.head()

,date,u_timestamp,price,target,timestamp,BITCOIN_NE.text_COUNT_last1days,BITCOIN_NE.text_last1days,text,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.electricity,VEC.billion loans,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,...,VEC.coinjoin,VEC.star,VEC.coindeal,VEC.congress,VEC.bitcoin mining,VEC.grayscale,VEC.printing,VEC.return,VEC.regulations,VEC.crypto traders,VEC.global economy,VEC.allowing,VEC.ceos,VEC.authorities,VEC.funding,VEC.plustoken,VEC.stocks,VEC.presidential,VEC.pool,VEC.billion,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.reports,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister
0,28-08-2020 17:00:00,1.598634e+09,11545.0,1,1577898000,3,[etoro ceo yoni assia on reaching 12 million u...,etoro ceo yoni assia on reaching 12 million us...,"{'neg': 0.0, 'neu': 0.954, 'pos': 0.046, 'comp...",0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,27-08-2020 17:00:00,1.598548e+09,11343.0,0,1577984400,7,[government confirms crypto profits not taxabl...,government confirms crypto profits not taxable...,"{'neg': 0.032, 'neu': 0.787, 'pos': 0.181, 'co...",0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,26-08-2020 17:00:00,1.598461e+09,11471.0,1,1578070800,8,[ticker tool uses the bch blockchain to provid...,ticker tool uses the bch blockchain to provide...,"{'neg': 0.122, 'neu': 0.785, 'pos': 0.093, 'co...",0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,25-08-2020 17:00:00,1.598375e+09,11341.0,0,1578157200,6,[crypto fundraising is changing again in 2020 ...,crypto fundraising is changing again in 2020 x...,"{'neg': 0.063, 'neu': 0.847, 'pos': 0.09, 'com...",0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,23-08-2020 17:00:00,1.598202e+09,11663.0,0,1578243600,5,[subhash chandra garg on the future of crypto ...,subhash chandra garg on the future of crypto x...,"{'neg': 0.049, 'neu': 0.772, 'pos': 0.179, 'co...",0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
df_anchor.to_csv("BTC_Final.csv")

In [ ]:
df_anchor["text_posneg"] = df_anchor["text_polarity"].apply(lambda x : 0 if x["neg"]==0.0 else x["pos"]/x["neg"]) 


In [ ]:
df_anchor["text_polarity"] = df_anchor["text_polarity"].apply(lambda x : x["compound"]) 

In [ ]:
df_final = df_anchor[df_anchor.columns[~df_anchor.columns.isin(['timestamp','price','BITCOIN_NE.text_COUNT_last1days','BITCOIN_NE.text_last1days','text','VEC.source'])]]

In [ ]:
corr_matrix

,u_timestamp,target,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.electricity,VEC.billion loans,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,VEC.supreme,VEC.merchant,VEC.pay,VEC.jim rogers,VEC.global,VEC.premium,...,VEC.star,VEC.coindeal,VEC.congress,VEC.bitcoin mining,VEC.grayscale,VEC.printing,VEC.return,VEC.regulations,VEC.crypto traders,VEC.global economy,VEC.allowing,VEC.ceos,VEC.authorities,VEC.funding,VEC.plustoken,VEC.stocks,VEC.presidential,VEC.pool,VEC.billion,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.reports,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.volumes,VEC.time,VEC.laws,VEC.expands,VEC.integrates,VEC.digital gold,VEC.minister,text_posneg
u_timestamp,1.000000,0.078198,0.129950,0.236463,0.024866,0.016966,0.123922,0.072524,0.124121,0.049183,0.004606,0.112906,0.010557,0.111211,0.091918,0.064726,0.129206,0.022020,0.011190,0.061034,0.004805,0.015151,0.019713,0.068820,0.056612,0.009435,0.006413,0.030510,0.082679,0.034443,0.078910,0.081417,0.036354,0.113708,0.200118,0.100072,0.012899,0.081853,0.014039,0.131730,...,0.084130,0.006629,0.028516,0.083581,0.154494,0.011443,0.057049,0.251854,0.054024,0.053511,0.043128,0.001228,0.143573,0.169558,0.044644,0.135567,0.003657,0.027061,0.060303,0.046854,0.195145,0.148480,0.069636,0.065393,0.045995,0.094241,0.138320,0.007658,0.015527,0.106169,0.025300,0.019824,0.029360,0.071739,0.101684,0.011814,0.012055,0.047789,0.125237,0.167387
target,0.078198,1.000000,0.003190,0.007029,0.085648,0.055801,0.029254,0.029254,0.012355,0.060324,0.051381,0.067167,0.073593,0.046614,0.055801,0.066350,0.010044,0.010044,0.068789,0.029742,0.031104,0.052473,0.007072,0.051335,0.007072,0.038423,0.055801,0.040764,0.055801,0.029254,0.105122,0.046614,0.029254,0.084338,0.068296,0.010044,0.044417,0.099792,0.096975,0.010044,...,0.029254,0.099792,0.071151,0.079022,0.014329,0.066350,0.029254,0.086700,0.075889,0.056393,0.010044,0.029254,0.029254,0.002510,0.122482,0.010044,0.071172,0.038423,0.079196,0.029254,0.027189,0.048335,0.040764,0.099792,0.122482,0.095636,0.055801,0.070726,0.055801,0.007072,0.046614,0.088523,0.014329,0.055189,0.018256,0.055801,0.018256,0.055801,0.055801,0.052693
text_polarity,0.129950,0.003190,1.000000,0.001274,0.107172,0.072686,0.011307,0.031112,0.050416,0.013261,0.051374,0.028992,0.088898,0.034650,0.027473,0.031299,0.115442,0.048378,0.069143,0.055117,0.053331,0.068775,0.108164,0.026941,0.033779,0.068736,0.121467,0.030974,0.026678,0.013059,0.012521,0.041678,0.030017,0.020595,0.103773,0.097116,0.048943,0.002063,0.054399,0.019057,...,0.036866,0.129198,0.140620,0.064293,0.010996,0.030241,0.080573,0.083699,0.114461,0.087168,0.010937,0.130504,0.097365,0.050936,0.024753,0.037619,0.067307,0.006983,0.015827,0.036654,0.162731,0.122871,0.072565,0.082240,0.056778,0.080615,0.034789,0.084038,0.051143,0.008020,0.027362,0.084689,0.023142,0.183783,0.108385,0.111943,0.187024,0.024956,0.133839,0.583386
text_subjectivity,0.236463,0.007029,0.001274,1.000000,0.025765,0.121733,0.038087,0.130127,0.018281,0.045671,0.079700,0.008682,0.019701,0.035977,0.106020,0.027202,0.009016,0.041041,0.015162,0.078416,0.065893,0.015234,0.046200,0.142424,0.012155,0.009560,0.023044,0.082170,0.067721,0.073942,0.043198,0.148080,0.062518,0.082274,0.065013,0.068379,0.055459,0.042369,0.033329,0.112529,...,0.008627,0.014743,0.022876,0.040411,0.022257,0.068423,0.067242,0.112477,0.069521,0.223544,0.048928,0.039827,0.019999,0.008237,0.029348,0.008468,0.015107,0.016672,0.023473,0.040126,0.105260,0.032222,0.022388,0.077800,0.093806,0.041788,0.065432,0.014311,0.026628,0.010297,0.062664,0.001272,0.110822,0.127241,0.017268,0.053682,0.013236,0.131765,0.044016,0.073185
VEC.block reward,0.024866,0.085648,0.107172,0.02576

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score# We drop correlated features because with so many binary 
# ngram variables there's a lot of noise

corr_matrix = df_final.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

In [ ]:
df_final.drop([column for column in upper.columns if any(upper[column] > 0.6)], axis=1, inplace=True)
# Now we separate in train/test sets
df_final.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


,date,u_timestamp,target,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,VEC.supreme,VEC.merchant,VEC.jim rogers,VEC.premium,VEC.hacked,VEC.alltime high,VEC.billionaire,...,VEC.ddos,VEC.file,VEC.sparks,VEC.usd,VEC.face,VEC.physical,VEC.money printing,VEC.48,VEC.45,VEC.42,VEC.40,VEC.41,VEC.tax bitcoin,VEC.holding,VEC.digital,VEC.developers,VEC.coinjoin,VEC.coindeal,VEC.grayscale,VEC.return,VEC.crypto traders,VEC.allowing,VEC.funding,VEC.plustoken,VEC.stocks,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.laws,VEC.digital gold,text_posneg
0,28-08-2020 17:00:00,1.598634e+09,1,0.2023,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
1,27-08-2020 17:00:00,1.598548e+09,0,0.9136,0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5.656250
2,26-08-2020 17:00:00,1.598461e+09,1,-0.5647,0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0.762295
3,25-08-2020 17:00:00,1.598375e+09,0,0.1531,0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.428571
4,23-08-2020 17:00:00,1.598202e+09,0,0.8126,0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.653061


In [ ]:
df_final

,date,u_timestamp,target,text_polarity,text_subjectivity,VEC.block reward,VEC.shop,VEC.code,VEC.virgin,VEC.chain,VEC.miner,VEC.demand,VEC.dollar,VEC.month,VEC.funding debate,VEC.kicks,VEC.httpbitcoincom wallet,VEC.battle,VEC.chair,VEC.issues,VEC.chinese,VEC.defi flash,VEC.group,VEC.countdown,VEC.list,VEC.charge,VEC.program,VEC.stimulus checks,VEC.helps,VEC.encrypted,VEC.breach,VEC.lawmakers,VEC.worth,VEC.supreme,VEC.merchant,VEC.jim rogers,VEC.premium,VEC.hacked,VEC.alltime high,VEC.billionaire,...,VEC.ddos,VEC.file,VEC.sparks,VEC.usd,VEC.face,VEC.physical,VEC.money printing,VEC.48,VEC.45,VEC.42,VEC.40,VEC.41,VEC.tax bitcoin,VEC.holding,VEC.digital,VEC.developers,VEC.coinjoin,VEC.coindeal,VEC.grayscale,VEC.return,VEC.crypto traders,VEC.allowing,VEC.funding,VEC.plustoken,VEC.stocks,VEC.sanctions,VEC.bch,VEC.assets,VEC.calls,VEC.tank,VEC.space,VEC.youtube,VEC.daily,VEC.2000,VEC.safe,VEC.heats,VEC.profits,VEC.laws,VEC.digital gold,text_posneg
0,28-08-2020 17:00:00,1.598634e+09,1,0.2023,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
1,27-08-2020 17:00:00,1.598548e+09,0,0.9136,0.462500,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,5.656250
2,26-08-2020 17:00:00,1.598461e+09,1,-0.5647,0.254167,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0.762295
3,25-08-2020 17:00:00,1.598375e+09,0,0.1531,0.518182,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.428571
4,23-08-2020 17:00:00,1.598202e+09,0,0.8126,0.319886,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3.653061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
231,05-01-2020 17:00:00,1.578244e+09,0,0.0000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
232,04-01-2020 17:00:00,1.578157e+09,0,0.0000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
233,03-01-2020 17:00:00,1.578071e+09,1,0.0000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000
234,02-01-2020 17:00:00,1.577984e+09,0,0.0000,0.000000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.000000


In [ ]:
X = df_final.iloc[:,3:].values
y = df_final.iloc[:,2].values

In [ ]:
div = int(round(len(X) * 0.2))
X_train = X[:div]
y_train = y[:div]
X_test = X[div:]
y_test = y[div:]
# Finally, we perform ML and see results

In [ ]:
X_train[:10]

array([[ 0.2023    ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.9136    ,  0.4625    ,  0.        , ...,  0.        ,
         0.        ,  5.65625   ],
       [-0.5647    ,  0.25416667,  0.        , ...,  0.        ,
         0.        ,  0.76229508],
       ...,
       [ 0.4588    ,  0.05      ,  0.        , ...,  0.        ,
         1.        ,  2.33333333],
       [ 0.2691    ,  0.48166667,  0.        , ...,  0.        ,
         0.        ,  1.28358209],
       [-0.7351    ,  0.70666667,  0.        , ...,  0.        ,
         0.        ,  0.47619048]])

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
forest = RandomForestClassifier(random_state = 0)

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
df_res = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})

In [ ]:
threshold = 0.5
preds = [1 if val > threshold else 0 for val in df_res['y_pred']]
print(metrics.confusion_matrix(preds, df_res['y_test']))
print('Accuracy Score:')
print(accuracy_score(preds, df_res['y_test']))
print('Precision Score:')
print(precision_score(preds, df_res['y_test']))

[[19 19]
 [73 78]]
Accuracy Score:
0.5132275132275133
Precision Score:
0.8041237113402062


In [ ]:
n_estimators = [200, 500, 1000]
max_depth = [5, 15, 25, 30]
min_samples_split = [2, 5, 10,]
min_samples_leaf = [1, 2, 5, 10] 

hyperF = dict(n_estimators = n_estimators, max_depth = max_depth,  
              min_samples_split = min_samples_split, 
             min_samples_leaf = min_samples_leaf)

gridF = GridSearchCV(forest, hyperF, cv = 3, verbose = 1, 
                      n_jobs = -1)
bestF = gridF.fit(X_train, y_train)

Fitting 3 folds for each of 144 candidates, totalling 432 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   29.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 432 out of 432 | elapsed:  4.5min finished


In [ ]:
bestF.best_params_

{'max_depth': 5,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 200}

In [ ]:
forest_opt = RandomForestClassifier(max_depth=5,min_samples_leaf=1,min_samples_split=2,n_estimators=200,random_state = 0)

In [ ]:
forest_opt.fit(X_train, y_train)
y_pred_opt = forest_opt.predict(X_test)
df_res_opt = pd.DataFrame({'y_test':y_test, 'y_pred_opt':y_pred_opt})

In [ ]:
print(metrics.confusion_matrix(df_res_opt['y_pred_opt'], df_res_opt['y_test']))
print('Accuracy Score:')
print(accuracy_score(df_res_opt['y_pred_opt'], df_res_opt['y_test']))
print('Precision Score:')
print(precision_score(df_res_opt['y_pred_opt'], df_res_opt['y_test']))

[[ 1  0]
 [91 97]]
Accuracy Score:
0.5185185185185185
Precision Score:
1.0
